In [55]:
# (c) 2016 Everaldo Aguiar & Reid Johnson
#
# Modified from:
# Marcel Caraciolo (https://gist.github.com/marcelcaraciolo/1423287)
#
# Functions to compute and extract association rules from a given frequent itemset 
# generated by the Apriori algorithm.
#
# The Apriori algorithm is defined by Agrawal and Srikant in:
# Fast algorithms for mining association rules
# Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994
import csv
import numpy as np


def apriori(dataset, min_support=0.5, verbose=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate 
        candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.

    References
    ----------
    .. [1] R. Agrawal, R. Srikant, "Fast Algorithms for Mining Association 
           Rules", 1994.

    """
    C1 = create_candidates(dataset)
    D = list(map(set, dataset))
    F1, support_data = support_prune(D, C1, min_support, verbose=False) # prune candidate 1-itemsets
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        Fk, supK = support_prune(D, Ck, min_support) # prune candidate itemsets
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    if verbose:
        # Print a list of all the frequent itemsets.
        for kset in F:
            for item in kset:
                print("" \
                    + "{" \
                    + "".join(str(i) + ", " for i in iter(item)).rstrip(', ') \
                    + "}" \
                    + ":  sup = " + str(round(support_data[item], 3)))

    return F, support_data

def create_candidates(dataset, verbose=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    #c1.sort()

    if verbose:
        # Print a list of all the candidate items.
        print("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    return list(map(frozenset, c1))

def support_prune(dataset, candidates, min_support, verbose=False):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    sscnt = {} # set for support counts
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset)) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    for key in sscnt:
        # Calculate the support of itemset key.
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if verbose:
        for kset in retlist:
            for item in kset:
                print("{" + str(item) + "}")
        print("")
        for key in sscnt:
            print("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets being evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            #a.sort()
            #b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return retList

def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.
    """
    m = len(H[0])
    if m == 1:
        Hmp1 = calc_confidence(freq_set, H, support_data, rules, min_confidence, verbose)
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1, support_data, rules, min_confidence, verbose)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence, verbose)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:
            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if verbose:
                print("" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence=0.5, verbose=True):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([itemset]) for itemset in freq_set]
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence, verbose)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, verbose)

    return rules

In [96]:
#Reading The CSV
import pandas as pd

data=pd.read_csv('crime.csv',engine='python')
data['SHOOTING']=data['SHOOTING'].fillna('N')
data = data.dropna()
data = data.drop(['INCIDENT_NUMBER', 'OFFENSE_CODE','OFFENSE_DESCRIPTION','REPORTING_AREA',
           'OCCURRED_ON_DATE','Lat','Long','Location','OCCURRED_ON_DATE','YEAR','SHOOTING'], axis=1)

data['MONTH']=data['MONTH'].map({1:'Jan', 2:'Feb', 
                                       3:'March', 4:'Apr', 
                                       5:'May', 6:'June', 
                                       7:'July', 8:'Aug', 
                                       9:'Sep', 10:'Oct', 
                                       11:'Nov', 12:'Dec'})
#Displaying the column names and their info for the data set
data['Time'] = 0
data['Time'].loc[(data['HOUR'] > 20)] = 'Night'
data['Time'].loc[(data['HOUR'] <= 3)] = 'Night'
data['Time'].loc[(data['HOUR'] > 3) & (data['HOUR'] <= 8)] = 'Early Morning'
data['Time'].loc[(data['HOUR'] > 8) & (data['HOUR'] <= 20)] = 'Day'
data.head()


,OFFENSE_CODE_GROUP,DISTRICT,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Time
0,Disorderly Conduct,E18,Oct,Wednesday,20,Part Two,ARLINGTON ST,Day
1,Property Lost,D14,Aug,Thursday,20,Part Three,ALLSTON ST,Day
2,Other,B2,Oct,Wednesday,19,Part Two,DEVON ST,Day
3,Aggravated Assault,A1,Oct,Wednesday,20,Part One,CAMBRIDGE ST,Day
4,Aircraft,A7,Oct,Wednesday,20,Part Three,PRESCOTT ST,Day


In [110]:
data['Time'].unique()


array(['Day', 'Early Morning', 'Night'], dtype=object)

In [99]:

#data = [data for data in data_iter]
dataset = np.asarray(data[1:])

D = list(map(set, dataset))

type(dataset)

numpy.ndarray

In [100]:
dataset.shape

(304658, 8)

In [101]:
dataset[0]

array(['Property Lost', 'D14', 'Aug', 'Thursday', 20, 'Part Three',
       'ALLSTON ST', 'Day'], dtype=object)

In [102]:
dataset[1]

array(['Other', 'B2', 'Oct', 'Wednesday', 19, 'Part Two', 'DEVON ST',
       'Day'], dtype=object)

In [103]:
type(D[0])

set

In [105]:
D[0]

{20,
 'ALLSTON ST',
 'Aug',
 'D14',
 'Day',
 'Part Three',
 'Property Lost',
 'Thursday'}

In [112]:
# Generate candidate itemsets.
C1 = create_candidates(dataset[:100000], verbose=True) # candidate 1-itemsets

{Property Lost, D14, Aug, Thursday, 20, Part Three, ALLSTON ST, Day, Other, B2, Oct, Wednesday, 19, Part Two, DEVON ST, Aggravated Assault, A1, Part One, CAMBRIDGE ST, Aircraft, A7, PRESCOTT ST, Vandalism, C11, Tuesday, DORCHESTER AVE, Verbal Disputes, TREMONT ST, Simple Assault, E18, AVILA RD, Towed, D4, COMMONWEALTH AVE, Motor Vehicle Accident Response, FOSTER ST, Auto Theft, E13, Monday, BYNNER ST, Medical Assistance, 17, BELNEL RD, Investigate Person, B3, 8, ELIZABETH ST, Early Morning, FOTTLER RD, 18, HALLET ST, DUNKELD ST, KILMARNOCK ST, Fire Related Reports, NAVILLUS TER, C6, COLUMBIA RD, Larceny, HUNTINGTON AVE, Investigate Property, HANSFORD ST, AKRON ST, Harassment, 0, MOUNT VERNON ST, Night, Drug Violation, BOYLSTON ST, May, Saturday, HARRISON AVE, MONTCALM AVE, MARGARET ST, CHAUCER ST, AMORY ST, GREENMOUNT ST, Violations, Sep, Sunday, WASHINGTON ST, Liquor Violation, DEARBORN ST, 22, GURNEY ST, 15, BOWDOIN ST, HYDE PARK AVE, Fraud, SPENCER ST, 7, LEYLAND ST, BUTLER ST, W BR

In [113]:
# Prune candidate 1-itemsets via support-based pruning to generate frequent 1-itemsets.
F1, support_data = support_prune(D[:100000], C1, 0.9, verbose=True)


{Property Lost}:  sup = 0.03499
{D14}:  sup = 0.06564
{Aug}:  sup = 0.08255
{Thursday}:  sup = 0.145
{20}:  sup = 0.05002
{Part Three}:  sup = 0.51277
{ALLSTON ST}:  sup = 0.00105
{Day}:  sup = 0.66377
{Other}:  sup = 0.06193
{B2}:  sup = 0.15128
{Oct}:  sup = 0.08782
{Wednesday}:  sup = 0.14619
{19}:  sup = 0.0536
{Part Two}:  sup = 0.29571
{DEVON ST}:  sup = 0.00135
{Aggravated Assault}:  sup = 0.02456
{A1}:  sup = 0.10801
{Part One}:  sup = 0.18726
{CAMBRIDGE ST}:  sup = 0.00489
{Aircraft}:  sup = 0.0001
{A7}:  sup = 0.04142
{PRESCOTT ST}:  sup = 0.00043
{Vandalism}:  sup = 0.04488
{C11}:  sup = 0.13317
{Tuesday}:  sup = 0.14548
{DORCHESTER AVE}:  sup = 0.0172
{Verbal Disputes}:  sup = 0.04728
{TREMONT ST}:  sup = 0.01526
{Simple Assault}:  sup = 0.05341
{E18}:  sup = 0.05842
{AVILA RD}:  sup = 8e-05
{Towed}:  sup = 0.03948
{D4}:  sup = 0.12896
{COMMONWEALTH AVE}:  sup = 0.01305
{Motor Vehicle Accident Response}:  sup = 0.1018
{FOSTER ST}:  sup = 0.00051
{Auto Theft}:  sup = 0.0137

{BACK ST}:  sup = 6e-05
{KERWIN ST}:  sup = 0.00025
{DENNIS ST}:  sup = 7e-05
{HIGHGATE ST}:  sup = 0.00017
{O'MEARA CT}:  sup = 6e-05
{CLAYBOURNE ST}:  sup = 0.00052
{CHAMPNEY ST}:  sup = 9e-05
{CHILTON RD}:  sup = 4e-05
{WALNUT ST}:  sup = 0.00037
{ARCOLA ST}:  sup = 5e-05
{RUGBY RD}:  sup = 0.00022
{W SIXTH ST}:  sup = 0.00016
{MINDEN ST}:  sup = 0.00022
{CITY SQ}:  sup = 0.00018
{ALTACREST RD}:  sup = 1e-05
{OSPREY WAY}:  sup = 0.00017
{SARGENT ST}:  sup = 0.0002
{JEROME ST}:  sup = 0.00013
{NEW ST}:  sup = 0.00019
{CHILD ST}:  sup = 0.00031
{HOWE ST}:  sup = 0.00023
{PRINCETON ST}:  sup = 0.0008
{FERNALD TER}:  sup = 0.0002
{MAGAZINE ST}:  sup = 0.00036
{HOLBORN ST}:  sup = 0.00037
{GREENOCK ST}:  sup = 0.00013
{LAURIAT ST}:  sup = 0.00029
{ISABELLA ST}:  sup = 7e-05
{ROWE ST}:  sup = 0.00014
{NIGHTINGALE ST}:  sup = 0.00062
{CRANSTON ST}:  sup = 7e-05
{CORTES ST}:  sup = 0.00042
{QUINT AVE}:  sup = 0.00022
{PACIFIC ST}:  sup = 2e-05
{WESTMORE RD}:  sup = 0.00033
{BURT ST}:  sup =

{SEAPORT LN}:  sup = 4e-05
{MARMION ST}:  sup = 5e-05
{WACHUSETT ST}:  sup = 0.00025
{CASS ST}:  sup = 9e-05
{KEYSTONE ST}:  sup = 0.00012
{ELBOW ST}:  sup = 4e-05
{FORSYTH ST}:  sup = 6e-05
{WIGGLESWORTH ST}:  sup = 4e-05
{HALLIDAY ST}:  sup = 4e-05
{OAKVIEW TER}:  sup = 3e-05
{COWPER ST}:  sup = 1e-05
{ROSLINDALE AVE}:  sup = 6e-05
{W RUTLAND SQ}:  sup = 3e-05
{ROCKVIEW ST}:  sup = 5e-05
{SPALDING ST}:  sup = 2e-05
{RICHFIELD PARK}:  sup = 1e-05
{POMFRET ST}:  sup = 1e-05
{SALCOMBE ST}:  sup = 5e-05
{PARTRIDGE ST}:  sup = 7e-05
{DORIS ST}:  sup = 6e-05
{HOLMAN ST}:  sup = 7e-05
{DUNNING WAY}:  sup = 1e-05
{ARBORETUM RD}:  sup = 7e-05
{DENBY RD}:  sup = 4e-05
{HESSTON TER}:  sup = 2e-05
{REDFIELD ST}:  sup = 5e-05
{RICHARD B. ROSS WAY}:  sup = 2e-05
{ROCKLEDGE ST}:  sup = 3e-05
{SAINT THERESA AVE}:  sup = 7e-05
{SPRINGDALE ST}:  sup = 2e-05
{MALCOLM RD}:  sup = 1e-05
{CITY HALL PZ}:  sup = 1e-05
{BURTON ST}:  sup = 3e-05
{JUNE ST}:  sup = 7e-05
{SHAFTER ST}:  sup = 0.00016
{NEWHALL PL

{CLOVER ST}:  sup = 2e-05
{ANNAVOY ST}:  sup = 1e-05
{STOCK ST}:  sup = 1e-05
{VICKSBURG ST}:  sup = 1e-05
{FORT AVENUE TER}:  sup = 1e-05
{GENEVA ST}:  sup = 3e-05
{EILEEN CIR}:  sup = 1e-05
{BRAVES FIELD WAY}:  sup = 1e-05
{PINGREE ST}:  sup = 1e-05
{ANDREWS ST}:  sup = 1e-05
{ENNIS RD}:  sup = 1e-05
{FIELD'S CT}:  sup = 2e-05
{AL ZWIERCAN WAY}:  sup = 1e-05
{MATTHEW ST}:  sup = 1e-05
{NOTT ST}:  sup = 1e-05
{OLMSTEAD ST}:  sup = 1e-05
{STURTEVANT ST}:  sup = 1e-05
{STOCKWELL ST}:  sup = 4e-05
{RHODA ST}:  sup = 2e-05
{CARYLL ST}:  sup = 1e-05
{QUAIL ST}:  sup = 1e-05
{CORCORAN DR}:  sup = 1e-05
{BEAVER PL}:  sup = 1e-05
{LEVERETT CIR}:  sup = 1e-05
{CENTRE STREET TER}:  sup = 2e-05
{CONSTITUTION PLZ}:  sup = 1e-05
{FENCOURT ST}:  sup = 1e-05
{WILLISTON RD}:  sup = 2e-05
{DEXTER ROW}:  sup = 2e-05
{FRANKLIN CT}:  sup = 1e-05
{BRIGHAM ST}:  sup = 2e-05
{VENTURA ST}:  sup = 1e-05
{BARKER ST}:  sup = 2e-05
{WIDETT CIR}:  sup = 1e-05
{BONAIR ST}:  sup = 1e-05
{HERBERT ST}:  sup = 1e-05
{

In [114]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset[:100000], min_support=0.1, verbose=True)

{Friday}:  sup = 0.154
{Sunday}:  sup = 0.127
{Sep}:  sup = 0.157
{Saturday}:  sup = 0.141
{Night}:  sup = 0.229
{Early Morning}:  sup = 0.108
{B3}:  sup = 0.115
{Monday}:  sup = 0.141
{Motor Vehicle Accident Response}:  sup = 0.102
{D4}:  sup = 0.129
{Tuesday}:  sup = 0.145
{C11}:  sup = 0.133
{Part One}:  sup = 0.187
{A1}:  sup = 0.108
{Part Two}:  sup = 0.296
{Wednesday}:  sup = 0.146
{B2}:  sup = 0.151
{Day}:  sup = 0.664
{Part Three}:  sup = 0.513
{Thursday}:  sup = 0.145
{Friday, Day}:  sup = 0.104
{Sep, Day}:  sup = 0.104
{Part Three, Night}:  sup = 0.115
{Motor Vehicle Accident Response, Part Three}:  sup = 0.102
{Tuesday, Day}:  sup = 0.101
{Part One, Day}:  sup = 0.123
{Wednesday, Day}:  sup = 0.102
{Part Two, Day}:  sup = 0.203
{Part Three, Day}:  sup = 0.335


In [118]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.1, verbose=True)

{Day} ---> {Friday}:  conf = 0.157, sup = 0.104
{Friday} ---> {Day}:  conf = 0.676, sup = 0.104
{Day} ---> {Sep}:  conf = 0.157, sup = 0.104
{Sep} ---> {Day}:  conf = 0.663, sup = 0.104
{Night} ---> {Part Three}:  conf = 0.504, sup = 0.115
{Part Three} ---> {Night}:  conf = 0.225, sup = 0.115
{Part Three} ---> {Motor Vehicle Accident Response}:  conf = 0.199, sup = 0.102
{Motor Vehicle Accident Response} ---> {Part Three}:  conf = 1.0, sup = 0.102
{Day} ---> {Tuesday}:  conf = 0.152, sup = 0.101
{Tuesday} ---> {Day}:  conf = 0.694, sup = 0.101
{Day} ---> {Part One}:  conf = 0.185, sup = 0.123
{Part One} ---> {Day}:  conf = 0.657, sup = 0.123
{Day} ---> {Wednesday}:  conf = 0.153, sup = 0.102
{Wednesday} ---> {Day}:  conf = 0.695, sup = 0.102
{Day} ---> {Part Two}:  conf = 0.306, sup = 0.203
{Part Two} ---> {Day}:  conf = 0.687, sup = 0.203
{Day} ---> {Part Three}:  conf = 0.505, sup = 0.335
{Part Three} ---> {Day}:  conf = 0.653, sup = 0.335


In [119]:
# (c) 2014 Reid Johnson
#
# Modified from:
# Eric Naeseth <eric@naeseth.com>
# (https://github.com/enaeseth/python-fp-growth/blob/master/fp_growth.py)
#
# A Python implementation of the FP-growth algorithm.

from collections import defaultdict, namedtuple
#from itertools import imap

__author__ = 'Eric Naeseth <eric@naeseth.com>'
__copyright__ = 'Copyright © 2009 Eric Naeseth'
__license__ = 'MIT License'

def fpgrowth(dataset, min_support=0.5, include_support=True, verbose=False):
    """Implements the FP-growth algorithm.

    The `dataset` parameter can be any iterable of iterables of items.
    `min_support` should be an integer specifying the minimum number of
    occurrences of an itemset for it to be accepted.

    Each item must be hashable (i.e., it must be valid as a member of a
    dictionary or a set).

    If `include_support` is true, yield (itemset, support) pairs instead of
    just the itemsets.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate 
        candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    include_support : bool
        Include support in output (default=False).

    References
    ----------
    .. [1] J. Han, J. Pei, Y. Yin, "Mining Frequent Patterns without Candidate 
           Generation," 2000.

    """

    F = []
    support_data = {}
    for k,v in find_frequent_itemsets(dataset, min_support=min_support, include_support=include_support, verbose=verbose):
        F.append(frozenset(k))
        support_data[frozenset(k)] = v

    # Create one array with subarrays that hold all transactions of equal length.
    def bucket_list(nested_list, sort=True):
        bucket = defaultdict(list)
        for sublist in nested_list:
            bucket[len(sublist)].append(sublist)
        return [v for k,v in sorted(bucket.items())] if sort else bucket.values()

    F = bucket_list(F)
    
    return F, support_data

def find_frequent_itemsets(dataset, min_support, include_support=False, verbose=False):
    """
    Find frequent itemsets in the given transactions using FP-growth. This
    function returns a generator instead of an eagerly-populated list of items.

    The `dataset` parameter can be any iterable of iterables of items.
    `min_support` should be an integer specifying the minimum number of
    occurrences of an itemset for it to be accepted.

    Each item must be hashable (i.e., it must be valid as a member of a
    dictionary or a set).

    If `include_support` is true, yield (itemset, support) pairs instead of
    just the itemsets.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate 
        candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    include_support : bool
        Include support in output (default=False).

    """
    items = defaultdict(lambda: 0) # mapping from items to their supports
    processed_transactions = []

    # Load the passed-in transactions and count the support that individual
    # items have.
    for transaction in dataset:
        processed = []
        for item in transaction:
            items[item] += 1
            processed.append(item)
        processed_transactions.append(processed)

    # Remove infrequent items from the item support dictionary.
    items = dict((item, support) for item, support in items.items()
        if support >= min_support)

    # Build our FP-tree. Before any transactions can be added to the tree, they
    # must be stripped of infrequent items and their surviving items must be
    # sorted in decreasing order of frequency.
    def clean_transaction(transaction):
        #transaction = filter(lambda v: v in items, transaction)
        transaction.sort(key=lambda v: items[v], reverse=True)
        return transaction

    master = FPTree()
    for transaction in map(clean_transaction, processed_transactions):
        master.add(transaction)

    support_data = {}
    def find_with_suffix(tree, suffix):
        for item, nodes in tree.items():
            support = float(sum(n.count for n in nodes)) / len(dataset)
            if support >= min_support and item not in suffix:
                # New winner!
                found_set = [item] + suffix
                support_data[frozenset(found_set)] = support
                yield (found_set, support) if include_support else found_set

                # Build a conditional tree and recursively search for frequent
                # itemsets within it.
                cond_tree = conditional_tree_from_paths(tree.prefix_paths(item),
                    min_support)
                for s in find_with_suffix(cond_tree, found_set):
                    yield s # pass along the good news to our caller

    if verbose:
        # Print a list of all the frequent itemsets.
        for itemset, support in find_with_suffix(master, []):
            print("" \
                + "{" \
                + "".join(str(i) + ", " for i in iter(itemset)).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(round(support_data[frozenset(itemset)], 3)))

    # Search for frequent itemsets, and yield the results we find.
    for itemset in find_with_suffix(master, []):
        yield itemset

class FPTree(object):
    """
    An FP tree.

    This object may only store transaction items that are hashable (i.e., all
    items must be valid as dictionary keys or set members).
    """

    Route = namedtuple('Route', 'head tail')

    def __init__(self):
        # The root node of the tree.
        self._root = FPNode(self, None, None)

        # A dictionary mapping items to the head and tail of a path of
        # "neighbors" that will hit every node containing that item.
        self._routes = {}

    @property
    def root(self):
        """The root node of the tree."""
        return self._root

    def add(self, transaction):
        """
        Adds a transaction to the tree.
        """

        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # There is already a node in this tree for the current
                # transaction item; reuse it.
                next_point.increment()
            else:
                # Create a new point and add it as a child of the point we're
                # currently looking at.
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the route of nodes that contain this item to include
                # our new node.
                self._update_route(next_point)

            point = next_point

    def _update_route(self, point):
        """Add the given node to the route through all nodes for its item."""
        assert self is point.tree

        try:
            route = self._routes[point.item]
            route[1].neighbor = point # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # First node for this item; start a new route.
            self._routes[point.item] = self.Route(point, point)

    def items(self):
        """
        Generate one 2-tuples for each item represented in the tree. The first
        element of the tuple is the item itself, and the second element is a
        generator that will yield the nodes in the tree that belong to the item.
        """
        for item in self._routes.keys():
            yield (item, self.nodes(item))

            
    def nodes(self, item):
        """
        Generates the sequence of nodes that contain the given item.
        """

        try:
            node = self._routes[item][0]
        except KeyError:
            return

        while node:
            yield node
            node = node.neighbor

    def prefix_paths(self, item):
        """Generates the prefix paths that end with the given item."""

        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path

        return (collect_path(node) for node in self.nodes(item))

    def inspect(self):
        print("Tree:")
        self.root.inspect(1)

        print("")
        print("Routes:")
        for item, nodes in self.items():
            print("  %r" % item)
            for node in nodes:
                print("    %r" % node)

    def _removed(self, node):
        """Called when `node` is removed from the tree; performs cleanup."""

        head, tail = self._routes[node.item]
        if node is head:
            if node is tail or not node.neighbor:
                # It was the sole node.
                del self._routes[node.item]
            else:
                self._routes[node.item] = self.Route(node.neighbor, tail)
        else:
            for n in self.nodes(node.item):
                if n.neighbor is node:
                    n.neighbor = node.neighbor # skip over
                    if node is tail:
                        self._routes[node.item] = self.Route(head, n)
                    break

def conditional_tree_from_paths(paths, min_support):
    """Builds a conditional FP-tree from the given prefix paths."""
    tree = FPTree()
    condition_item = None
    items = set()

    # Import the nodes in the paths into the new tree. Only the counts of the
    # leaf notes matter; the remaining counts will be reconstructed from the
    # leaf counts.
    for path in paths:
        if condition_item is None:
            condition_item = path[-1].item

        point = tree.root
        for node in path:
            next_point = point.search(node.item)
            if not next_point:
                # Add a new node to the tree.
                items.add(node.item)
                count = node.count if node.item == condition_item else 0
                next_point = FPNode(tree, node.item, count)
                point.add(next_point)
                tree._update_route(next_point)
            point = next_point

    assert condition_item is not None

    # Calculate the counts of the non-leaf nodes.
    for path in tree.prefix_paths(condition_item):
        count = path[-1].count
        for node in reversed(path[:-1]):
            node._count += count

    # Eliminate the nodes for any items that are no longer frequent.
    for item in items:
        support = sum(n.count for n in tree.nodes(item))
        if support < min_support:
            # Doesn't make the cut anymore
            for node in tree.nodes(item):
                if node.parent is not None:
                    node.parent.remove(node)

    # Finally, remove the nodes corresponding to the item for which this
    # conditional tree was generated.
    for node in tree.nodes(condition_item):
        if node.parent is not None: # the node might already be an orphan
            node.parent.remove(node)

    return tree

class FPNode(object):
    """A node in an FP tree."""

    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None

    def add(self, child):
        """Adds the given FPNode `child` as a child of this node."""

        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")

        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self

    def search(self, item):
        """
        Checks to see if this node contains a child node for the given item.
        If so, that node is returned; otherwise, `None` is returned.
        """

        try:
            return self._children[item]
        except KeyError:
            return None

    def remove(self, child):
        try:
            if self._children[child.item] is child:
                del self._children[child.item]
                child.parent = None
                self._tree._removed(child)
                for sub_child in child.children:
                    try:
                        # Merger case: we already have a child for that item, so
                        # add the sub-child's count to our child's count.
                        self._children[sub_child.item]._count += sub_child.count
                        sub_child.parent = None # it's an orphan now
                    except KeyError:
                        # Turns out we don't actually have a child, so just add
                        # the sub-child as our own child.
                        self.add(sub_child)
                child._children = {}
            else:
                raise ValueError("that node is not a child of this node")
        except KeyError:
            raise ValueError("that node is not a child of this node")

    def __contains__(self, item):
        return item in self._children

    @property
    def tree(self):
        """The tree in which this node appears."""
        return self._tree

    @property
    def item(self):
        """The item contained in this node."""
        return self._item

    @property
    def count(self):
        """The count associated with this node's item."""
        return self._count

    def increment(self):
        """Increments the count associated with this node's item."""
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1

    @property
    def root(self):
        """True if this node is the root of a tree; false if otherwise."""
        return self._item is None and self._count is None

    @property
    def leaf(self):
        """True if this node is a leaf in the tree; false if otherwise."""
        return len(self._children) == 0

    def parent():
        doc = "The node's parent."
        def fget(self):
            return self._parent
        def fset(self, value):
            if value is not None and not isinstance(value, FPNode):
                raise TypeError("A node must have an FPNode as a parent.")
            if value and value.tree is not self.tree:
                raise ValueError("Cannot have a parent from another tree.")
            self._parent = value
        return locals()
    parent = property(**parent())

    def neighbor():
        doc = """
        The node's neighbor; the one with the same value that is "to the right"
        of it in the tree.
        """
        def fget(self):
            return self._neighbor
        def fset(self, value):
            if value is not None and not isinstance(value, FPNode):
                raise TypeError("A node must have an FPNode as a neighbor.")
            if value and value.tree is not self.tree:
                raise ValueError("Cannot have a neighbor from another tree.")
            self._neighbor = value
        return locals()
    neighbor = property(**neighbor())

    @property
    def children(self):
        """The nodes that are children of this node."""
        return tuple(self._children.values())
        
    def inspect(self, depth=0):
        print(('  ' * depth) + repr(self))
        for child in self.children:
            child.inspect(depth + 1)

    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)

def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.
    """
    m = len(H[0])
    if m == 1:
        Hmp1 = calc_confidence(freq_set, H, support_data, rules, min_confidence, verbose)
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1,  support_data, rules, min_confidence, verbose)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence, verbose)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:
            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if verbose:
                print("" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence=0.5, verbose=True):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([item]) for item in freq_set]
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence, verbose)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, verbose)

    return rules


In [122]:
# Generate all the frequent itemsets using the FP-growth algorithm.

F, support_data = fpgrowth(dataset, min_support=0.1,include_support = True, verbose=True)

{Day}:  sup = 0.663
{Part Three}:  sup = 0.493
{Day, Part Three}:  sup = 0.32
{Thursday}:  sup = 0.146
{Aug}:  sup = 0.107
{Part Two}:  sup = 0.305
{Day, Part Two}:  sup = 0.21
{B2}:  sup = 0.155
{Day, B2}:  sup = 0.102
{Wednesday}:  sup = 0.147
{Day, Wednesday}:  sup = 0.102
{Part One}:  sup = 0.198
{Day, Part One}:  sup = 0.131
{A1}:  sup = 0.107
{Tuesday}:  sup = 0.145
{Day, Tuesday}:  sup = 0.101
{C11}:  sup = 0.138
{D4}:  sup = 0.13
{Motor Vehicle Accident Response}:  sup = 0.102
{Part Three, Motor Vehicle Accident Response}:  sup = 0.102
{Monday}:  sup = 0.143
{B3}:  sup = 0.114
{Early Morning}:  sup = 0.106
{Night}:  sup = 0.231
{Part Three, Night}:  sup = 0.113
{Saturday}:  sup = 0.14
{Sunday}:  sup = 0.127
{Sep}:  sup = 0.103
{Friday}:  sup = 0.152
{Day, Friday}:  sup = 0.101
{July}:  sup = 0.105


In [124]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.1, verbose=True)

{Day} ---> {Part Three}:  conf = 0.482, sup = 0.32
{Part Three} ---> {Day}:  conf = 0.649, sup = 0.32
{Day} ---> {Part Two}:  conf = 0.316, sup = 0.21
{Part Two} ---> {Day}:  conf = 0.687, sup = 0.21
{Day} ---> {B2}:  conf = 0.154, sup = 0.102
{B2} ---> {Day}:  conf = 0.656, sup = 0.102
{Day} ---> {Wednesday}:  conf = 0.154, sup = 0.102
{Wednesday} ---> {Day}:  conf = 0.693, sup = 0.102
{Day} ---> {Part One}:  conf = 0.197, sup = 0.131
{Part One} ---> {Day}:  conf = 0.662, sup = 0.131
{Day} ---> {Tuesday}:  conf = 0.152, sup = 0.101
{Tuesday} ---> {Day}:  conf = 0.694, sup = 0.101
{Part Three} ---> {Motor Vehicle Accident Response}:  conf = 0.208, sup = 0.102
{Motor Vehicle Accident Response} ---> {Part Three}:  conf = 1.0, sup = 0.102
{Night} ---> {Part Three}:  conf = 0.489, sup = 0.113
{Part Three} ---> {Night}:  conf = 0.229, sup = 0.113
{Day} ---> {Friday}:  conf = 0.153, sup = 0.101
{Friday} ---> {Day}:  conf = 0.667, sup = 0.101
